In [1]:
# text = [ ['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'] , ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'] ]
# 1인덱스에 인풋, 2인덱스에 아웃풋  0 부정 / 1긍정 리뷰   

In [2]:
# 기본 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 다운, import, 리드
! pip install konlpy
import codecs
with codecs.open('data2.txt',encoding='utf-8') as f:
    data = [문장.split('\t') for 문장 in f.read().splitlines()]  

# 데이터 모양 확인
data[:10]  # 데이터 모양 확인
data = data[1:]  # 0인덱 필요없으니 제외하고 슬라이스 

# 긍정문장과 부정문장 스플릿
긍정문장_data = [  문장[1] for 문장 in data if 문장[2]=='1'  ] # 1:긍정리뷰 아웃풋     
부정문장_data = [  문장[1] for 문장 in data if 문장[2]=='0'  ] # 0:부정리뷰 아웃풋     
 
# 문장을 받으면 '단어/형태소'로 쪼개고 리스트화 하는 def 함수 선언
from konlpy.tag import Okt # 한국어 단어 토크나이저
from tqdm import tqdm # 무한루프에 걸렸는지 진행 중인지 확인하기 위한 # 작업의 진행사항을 알려주는 패키지  ~% 진행 표시
from nltk.util import ngrams # 엔그램 단어 묶기, 문맥
tk = Okt()
def 문장을받으면단어와형태소로쪼개는함수(문장):
  리턴단어형태소 =['/'.join(i) for i in tk.pos(문장)] #pos : [ ('단어','형태소') ] 여기서 i에  ( . )튜플들어가고 조인()에들어가면 단어,형태소 사이 쉼표를 /로 조인
  return 리턴단어형태소
문장을받으면단어와형태소로쪼개는함수(긍정문장_data[0]) # ['흠/Noun', '.../Punctuation', '포스터/Noun' ]

# 긍정문을 활용한 엔그램 리스트 생성
긍정문엔그램리스트 = []
for 문장 in tqdm(긍정문장_data): 
  긍정단어와형태소 = 문장을받으면단어와형태소로쪼개는함수(문장)  # tk_f() 는 def로  ['흠/Noun', '.../Punctuation', '포스터/Noun' ] 단어/형태소로 뽑는 함수
  ng = ngrams(긍정단어와형태소, 2, pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol='SE') # ng은 zip형태니까
  # ng출력 = [('SS', '흠/Noun'), ('포스터/Noun', 'SE')]
  긍정문엔그램리스트 += [t for t in ng] # 리스트형태로 뽑아줌 [('SS', '흠/Noun'), ('포스터/Noun', 'SE')] 타입만 list로 바뀌고 다른 건 같음
긍정문엔그램리스트  
     
# 부정문을 활용한 엔그램 리스트 생성
부정문엔그램리스트 = []
for 문장 in tqdm(부정문장_data):
  부정단어와형태소 = 문장을받으면단어와형태소로쪼개는함수(문장)
  ng = ngrams(부정단어와형태소, 2, pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol='SE')
  부정문엔그램리스트 += [t for t in ng]
부정문엔그램리스트

# cpd긍정 과 cpd부정 생성
# 긍정문엔그램리스트 = [('SS', '흠/Noun'), ('포스터/Noun', 'SE')] #긍정문
# 부정문엔그램리스트 = [('SS', '흠/Noun'), ('포스터/Noun', 'SE')] #부정문
from nltk import ConditionalFreqDist 
from nltk.probability import ConditionalProbDist,MLEProbDist # MLEProbDist 최대우도추정
cfd긍정 = ConditionalFreqDist(긍정문엔그램리스트) # cfd 다음단어조건부빈도
cpd긍정 = ConditionalProbDist(cfd긍정, MLEProbDist) # cpd 다음단어조건부확률
cfd부정 = ConditionalFreqDist(부정문엔그램리스트)
cpd부정 = ConditionalProbDist(cfd부정, MLEProbDist)
# cpd 는 긍정문장기준 cpd1 은 부정문장기준
# 랜덤문장생성기, 긍정문장생성기, 부정문장생성기를 위한 준비 완료

#긍정문장랜덤생성기
st긍정 = 'SS'
랜덤긍정단어 = []
import random
#random.seed(8) # 랜덤시드빼면 실행할 때마다 랜덤
while True:
  st긍정 = cpd긍정[st긍정].generate()  # SS다음 나올 단어를 gen으로 그냥 random으로 뽑고 
  if st긍정 == "SE" : # SE가 뽑힐 때까지 반복 (SS와 SE는 뽑히지 않음)
    break
  d = st긍정.split('/')[0]  # 스플릿후[0]인덱이면 단어를 
  랜덤긍정단어.append(d)  # 리스트에 넣음
#  랜덤긍정단어 = ['영상','진짜']  # 단어 하나하나 
긍정문장랜덤생성기 = ' '.join(랜덤긍정단어)  #  리스트에 들어간 단어 하나하나를 띄어쓰기 기준으로 조인
긍정문장랜덤생성기  

#부정문장랜덤생성기
st부정 = 'SS'
랜덤부정단어 = []
import random
#random.seed(8) # 랜덤시드빼면 실행할 때마다 랜덤
while True:
  st부정 = cpd부정[st부정].generate()  # SS다음 나올 단어를 gen으로 그냥 random으로 뽑고 
  if st부정 == "SE" : # SE가 뽑힐 때까지 반복 (SS와 SE는 뽑히지 않음)
    break
  d = st부정.split('/')[0]  # 스플릿후[0]인덱이면 단어를 
  랜덤부정단어.append(d)  # 리스트에 넣음
#  랜덤부정단어 = ['영상','진짜']  # 단어 하나하나
부정문장랜덤생성기 = ' '.join(랜덤부정단어)  #  리스트에 들어간 단어 하나하나를 띄어쓰기 기준으로 조인
부정문장랜덤생성기  

# 긍정문장랜덤생성기 + 부정문장랜덤생성기 def로 한번에 만들기
tk.tagset # 형태소 확인
def 긍정부정랜덤문장생성기(c="SS",cpd = cpd긍정): # 'SS' 8 cpd 는 디폴트값 # (,n_seed=8) 시드 조절하고 싶을 때
  #random.seed(n_seed)
  랜덤단어모음 = []
  while True:
    단어와형태소 = cpd[c].generate()  #  랜덤으로 나오는 (단어/형태소)
    if 단어와형태소 == "SE": # SE나오면 브레이크
      break
    단어 = 단어와형태소.split('/')[0] # 단어
    형태소 = 단어와형태소.split('/')[1] # 형태소
    if c=="SS":
      랜덤단어모음.append(단어.title()) #.title()첫문자만 대문자화
    elif c in ["'","\"",'(']:
      랜덤단어모음.append(단어)
    elif 단어와형태소 in ["'",",",".",")",":",";","?"]:
      랜덤단어모음.append(단어)
    elif 형태소 in ["Josa", "Punctuation", "Suffix"]:
      랜덤단어모음.append(단어)
    elif 단어와형태소 in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
               "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
      랜덤단어모음.append(단어)
    else:
      랜덤단어모음.append(" "+단어)
    c = 단어와형태소 # c는 첫단어고 단어와형태소는 뒤단어인데 c로 넣어줘서 루프를 SE 가나올 때까지 돌린다
  return ''.join(랜덤단어모음)
긍정부정랜덤문장생성기() #긍정생성기
긍정부정랜덤문장생성기(cpd=cpd부정) #부정생성기

# 생성기 모음
긍정문장랜덤생성기  
부정문장랜덤생성기  
긍정부정랜덤문장생성기() #긍정생성기
긍정부정랜덤문장생성기(cpd=cpd부정) #부정생성기

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 49.8 MB/s 


100%|██████████| 75173/75173 [04:06<00:00, 305.40it/s]


'보다가 뛰어나갈 뻔했고 재미는 악다구니 뿐.. 평점 조작 해서 극장판 때문에 넣고 다니며 뭔 그래픽의 빈약함. 정치적인데. 오직 이 될듯'

In [3]:
긍정문장랜덤생성기  

"진정한 사랑 이야기 '!"

In [4]:
부정문장랜덤생성기  

'리메이크 작'

In [5]:
긍정부정랜덤문장생성기() #긍정생성기 

'성룡을 깨고 연못으로 인간들 가득 차..'

In [6]:
긍정부정랜덤문장생성기(cpd=cpd부정) #부정생성기

'이런 영화 영화'